In [1]:
import csv
from getpass import getpass
from time import sleep
from datetime import datetime
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import Chrome
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
import warnings
warnings.filterwarnings('ignore')
import time
import pandas as pd
import pytz

In [ ]:
driver = Chrome('../../ML/datasets/chromedriver.exe')

In [ ]:
driver.get('https://www.twitter.com/login')

In [ ]:
username = driver.find_element_by_xpath('/html/body/div/div/div/div/main/div/div/div/div[2]/div[2]/div[1]/div/div/div[5]/label/div/div[2]/div/input')
username.send_keys('alapparate')
driver.find_element_by_xpath('//*[@id="react-root"]/div/div/div/main/div/div/div/div[2]/div[2]/div[1]/div/div/div[6]/div').click()

In [ ]:
mypass = getpass()

In [ ]:
password = driver.find_element_by_xpath('//*[@id="react-root"]/div/div/div/main/div/div/div/div[2]/div[2]/div[1]/div/div/div[3]/div/label/div/div[2]/div[1]/input')
password.send_keys(mypass)
driver.find_element_by_xpath('//*[@id="react-root"]/div/div/div/main/div/div/div/div[2]/div[2]/div[2]/div/div/div[1]/div/div/div').click()

In [ ]:
search_box = driver.find_element_by_xpath('//input[@aria-label="Search query"]')
search_box.send_keys('#nifty')
search_box.send_keys(Keys.RETURN)

In [ ]:
driver.find_element_by_link_text('Latest').click()

In [ ]:
cards = driver.find_elements_by_xpath('//article[@data-testid="tweet"]')

In [ ]:
card = cards[0]

In [ ]:
time_stamp = card.find_element_by_xpath('.//time')

In [ ]:
time_stamp

In [ ]:
tweet_text = card.find_element_by_xpath('./div/div/div/div[2]/div[2]/div[2]/div[1]/div').text

In [ ]:
tweet_text

In [ ]:
def get_tweet_data(card):
    """Extract data from tweet"""
    tweet_text = card.find_element_by_xpath('./div/div/div/div[2]/div[2]/div[2]/div[1]/div').text
    time_stamp = card.find_element_by_xpath('.//time').get_attribute('datetime')
    tweet = (tweet_text,time_stamp)
    return tweet

In [ ]:
get_tweet_data(card)

In [ ]:
tweet_data = []
for card in cards:
    data = get_tweet_data(card)
    tweet_data.append(data)

In [ ]:
tweet_data

In [ ]:
driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')

## Final Function

##### Initiate Chrome 

In [2]:
driver = Chrome('../../ML/datasets/chromedriver_old.exe')

##### Login

In [3]:
driver.get('https://www.twitter.com/login')
driver.maximize_window()
time.sleep(5)
username = driver.find_element_by_xpath('//input[@autocomplete="username"]')
myuname = getpass('enter username : ')
username.send_keys(myuname)
driver.find_element_by_xpath('/html/body/div/div/div/div[1]/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[1]/div/div/div[6]/div/span').click()
mypass = getpass('enter password : ')
password = driver.find_element_by_xpath('//input[@autocomplete="current-password"]')
password.send_keys(mypass)
driver.find_element_by_xpath('/html/body/div/div/div/div[1]/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[2]/div/div[1]/div/div/div/span').click()

enter username : ········
enter password : ········


##### Date for tweet extraction

In [10]:
start = input('start (YYYY-MM-DD) : ')
end = input('end (YYYY-MM-DD) : ')
search_str = f"(#nifty) until:{end} since:{start} -filter:links -filter:replies"
search_str

start (YYYY-MM-DD) : 2022-01-24
end (YYYY-MM-DD) : 2022-01-25


'(#nifty) until:2022-01-25 since:2022-01-24 -filter:links -filter:replies'

##### Searching the query

In [14]:
search_box = driver.find_element_by_xpath('//input[@aria-label="Search query"]')
search_box.send_keys("")
search_box.send_keys(search_str)
search_box.send_keys(Keys.RETURN)
driver.find_element_by_link_text('Latest').click()    #navigate to latest tab

##### get all tweets on page

###### func to get tweet text and timestamp

In [15]:
def get_tweet_data(card):
    """Extract data from tweet"""
    tweet_text = card.find_element_by_xpath('./div/div/div/div[2]/div[2]/div[2]/div[1]/div').text
    
    try :
            time_stamp = card.find_element_by_xpath('//div/div/div/div[2]/div[2]/div[1]/div/div/div[1]/a/time').get_attribute('datetime')
    except NoSuchElementException:
        return
    tweet = (tweet_text,time_stamp)
    return tweet

In [21]:
data = []
tweet_ids = set()
last_pos = driver.execute_script('return window.pageYOffset;')
scrolling = True

while scrolling:
    page_cards = driver.find_elements_by_xpath('//article[@data-testid="tweet"]')
    for card in page_cards[-25:]:
        tweet = get_tweet_data(card)
        if bool(tweet):
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)
                
    scroll_attempt = 0
    while True:
        # check scroll pos
        driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
        sleep(2)
        curr_pos = driver.execute_script('return window.pageYOffset;')
        if last_pos == curr_pos:
            scroll_attempt +=1
            driver.execute_script('window.scrollTo(0, 0);')
            driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')

            # end scroll region
            if scroll_attempt >=5:
                scrolling = False
                break
            else:
                sleep(2)
        else:
            last_pos=curr_pos
            break

In [22]:
df = pd.DataFrame(columns=['tweet','time'])
df

,tweet,time


In [23]:
for d in data:
    tw, ti = d[0],d[1]
    df = df.append({'tweet':tw,'time':ti},ignore_index=True)

In [24]:
df

,tweet,time
0,#Sensex Down 634.20 Points (1.06%) to close at...,2022-01-20T18:20:15.000Z
1,Abhi market vo mode per he ki koi clearly nhi ...,2022-01-20T18:20:15.000Z
2,Kind of person you will become in five years w...,2022-01-20T18:20:15.000Z
3,Koi HUL & wipro khreed be raha hai ?\nYa fir m...,2022-01-20T18:20:15.000Z
4,ITC Q3 RESULT\n\nCo. To announce Q3 Result On ...,2022-01-20T18:20:15.000Z
...,...,...
776,#nifty #trading\nSome rule I am applying to my...,2022-01-20T02:15:30.000Z
777,Market view for today 20/01/22\nNifty fut supp...,2022-01-20T02:15:30.000Z
778,Patience is a key skill of good trader . \n#ni...,2022-01-20T02:15:30.000Z
779,#NIFTY#BANK NIFTY #U.S. BOND YIELDS NOW START ...,2022-01-20T02:15:30.000Z


In [25]:
df.to_csv(f'tweets_{start}_{end}.csv',index=False)

##### Converting Time Zone to India

In [ ]:
# df['time'][0]